# Reconnaissance du type de cuisine d'une rectte à partir des ingrédients

Imaginons un site de partage de recette communautaire désirant proposer une fonction de tag automatique des recettes suivant le type de cuisine. Pour mettre au point cette fonction, nous allons utiliser un classifieur et un ensemble de recette préalablement étiqueté.

In [1]:
import pandas as pd
import numpy as np
import json

Lecture des données et mise en forme. La première colones contient des données vides et la dernière "cuisine" contient les étiquettes.

In [75]:
with open("./recipes_train.json") as f:
    data_train=json.load(f)
    y = np.array([recipe["cuisine"] for recipe in data_train])
    xtext = [recipe["ingredients"] for recipe in data_train]

In [76]:
ingredients = np.unique(np.concatenate(xtext))

In [44]:
dict_ingredients = dict((ingredients[i],i) for i in range(0, len(ingredients)))

In [60]:
def encode_one_hot(recipe):
    x = np.zeros((1,len(ingredients)))
    indices = [dict_ingredients[ing] for ing in recipe]
    x[0,indices]=1
    return x
X = np.vstack([encode_one_hot(recipes) for recipes in xtraintext])

In [78]:
sample = np.random.permutation(X.shape[0])[:20000]
Xs=X[:,np.sum(X,axis=0)>200]
Xs=Xs[sample,:]
ys=y[sample]

In [79]:
select_ingredients=ingredients[np.sum(X,axis=0)>200]
select_ingredients

array(['Shaoxing wine', 'Sriracha', 'active dry yeast',
       'all-purpose flour', 'andouille sausage', 'apple cider vinegar',
       'arborio rice', 'asparagus', 'avocado', 'baby spinach', 'bacon',
       'bacon slices', 'baguette', 'baking potatoes', 'baking powder',
       'baking soda', 'balsamic vinegar', 'bananas', 'basil',
       'basil leaves', 'basmati rice', 'bay leaf', 'bay leaves',
       'beansprouts', 'beef', 'beef broth', 'beer', 'bell pepper',
       'black beans', 'black olives', 'black pepper', 'black peppercorns',
       'boiling water', 'boneless chicken skinless thigh',
       'boneless skinless chicken breast halves',
       'boneless skinless chicken breasts', 'bread crumbs', 'broccoli',
       'brown rice', 'brown sugar', 'butter', 'buttermilk', 'cabbage',
       'cajun seasoning', 'cannellini beans', 'canola oil', 'capers',
       'carrots', 'cashew nuts', 'cauliflower', 'cayenne',
       'cayenne pepper', 'celery', 'celery ribs', 'cheddar cheese',
       'che

Les variables correspondent chacune à un ingrédient et la présence/l'absence de celui-ci dans une recette est codé à l'aide de 1 / 0.

Les données ont été légérement pré-traitée et seul les 977 ingrédients les plus courants ont été conservés. La matrice de données contient donc in-fine 39 774 recettes et 977 ingrédients.

Enfin, le classifieur devra reconaitre 20 type différents de cuisine. Ce jeu de données n'est donc pas petit et commence à être de taille intéressante.

In [88]:
np.unique(ys)

array(['brazilian', 'british', 'cajun_creole', 'chinese', 'filipino',
       'french', 'greek', 'indian', 'irish', 'italian', 'jamaican',
       'japanese', 'korean', 'mexican', 'moroccan', 'russian',
       'southern_us', 'spanish', 'thai', 'vietnamese'], dtype='<U12')

Comme précedement, nous allons importer des fonctions utiles de scikit-learn et préparer un jeu de données d'apprentissage et un jeu de données de test.'

In [82]:
from sklearn.model_selection import train_test_split,cross_val_score
# from sklearn.cross_validation import train_test_split,cross_val_score v0.15
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.3, random_state=0)

Nous avons choisi d'utiliser des random forest pour résoudre ce problème. Estimons l'erreur de généralisation au sens du f-score par validation croisée.

In [83]:
clf = RandomForestClassifier(n_estimators=20)
scores = cross_val_score(clf, X_train, y_train, scoring='accuracy')
scores.mean()  

0.6076428571428572

Regardons ce qui se passe sur le jeu de test.

In [84]:
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [85]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   brazilian       0.28      0.24      0.26        63
     british       0.23      0.19      0.21       117
cajun_creole       0.68      0.55      0.61       232
     chinese       0.63      0.79      0.70       411
    filipino       0.47      0.41      0.44       106
      french       0.38      0.34      0.36       393
       greek       0.66      0.51      0.57       173
      indian       0.75      0.82      0.78       462
       irish       0.39      0.20      0.26       116
     italian       0.65      0.79      0.71      1189
    jamaican       0.54      0.18      0.27        78
    japanese       0.51      0.43      0.46       214
      korean       0.63      0.35      0.45       123
     mexican       0.77      0.85      0.81       998
    moroccan       0.56      0.36      0.44       116
     russian       0.39      0.10      0.15        73
 southern_us       0.46      0.56      0.51       636
     spanish       0.39    

In [90]:
cm=pd.DataFrame(confusion_matrix(y_pred,y_test), index=np.sort(np.unique(ys)))
cm.columns=np.sort(np.unique(ys))
cm

,brazilian,british,cajun_creole,chinese,filipino,french,greek,indian,irish,italian,jamaican,japanese,korean,mexican,moroccan,russian,southern_us,spanish,thai,vietnamese
brazilian,15,6,1,1,2,1,2,3,1,3,2,1,0,6,0,0,6,0,2,1
british,2,22,2,2,1,15,1,2,9,11,1,2,0,1,6,2,16,1,0,0
cajun_creole,1,1,128,0,1,5,1,1,0,7,3,2,0,3,1,0,32,2,0,0
chinese,0,4,2,326,19,9,0,6,1,6,6,41,33,3,0,0,10,1,35,14
filipino,2,0,0,4,43,0,0,2,2,3,5,4,2,3,0,0,5,1,9,6
french,1,11,8,4,5,133,7,6,20,62,0,6,1,10,3,11,45,15,1,0
greek,0,2,2,1,1,3,88,3,0,20,0,1,1,2,4,1,1,4,0,0
indian,0,3,0,1,3,5,3,378,0,9,8,23,3,16,19,0,12,4,12,5
irish,0,4,0,2,3,4,1,2,23,4,1,1,1,3,0,2,5,3,0,0
italian,8,14,22,7,3,144,50,9,17,941,6,7,5,46,13,17,81,54,7,4


# Questions ?
## 1) Qu'elle est le taux de bonne classification de ce classifieur sur les données d'apprentissage, estimé par validation croisée (5 folds) et enfin sur les données de test ? 
Vous pourrez en profiter pour allez lire l'aide de la fonction sklearn.metrics.accuracy_score.
## Qu'en concluez-vous ?


## 2) Essayer d'améliorer les performances de ce classifieur en terme de taux de bonne classification ?
Vous pourrez faire varier le nombre d'arbre entre [20,50,100,200]. Servez-vous de la fonction GridSearchCV vue dans le premier notebbok.

In [46]:
from sklearn.model_selection import GridSearchCV

## 2) Tester une autre solution (Regression logistique)
Vous pourrez avantageusement lire la documentation de sklearn.linear_model.LogisticRegression

In [47]:
from sklearn.linear_model import LogisticRegression

## 3) Essayer de mettre en place une solution de rejet, pour obtenir un taux de bonne classification d'au moins 90% ? 
Pour ce faire vous utiliserez les probabilités fournies par le classifieur et ne prendrait une décision que quand ces probabilités sont supérieurs à un certain seuil (à déterminer). La fonction predict_proba devrait pouvoir vous aider.